In [417]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [418]:
def getTestandTrainSplit():
    
    iris_df_train =  pd.read_csv(r'C:\Users\rajes\OneDrive\Documents\AMPBA\DL\iris_training.csv', header=None)
    iris_df_test =  pd.read_csv(r'C:\Users\rajes\OneDrive\Documents\AMPBA\DL\iris_test.csv', header=None)

    x_train = iris_df_train.iloc[:,0:4].values
    y_train = iris_df_train.iloc[:,-1].values

    x_test = iris_df_test.iloc[:,0:4].values
    y_test = iris_df_test.iloc[:,-1].values
    
    return x_train,y_train, x_test,y_test


##### 1. Create a simple SoftMax classifier with 3 neurons in output layer and no hidden layer using TensorFlow Core APIs. Save it as iris_softmax.py.


In [419]:
x_train,y_train, x_test,y_test = getTestandTrainSplit()

In [420]:
y_train = iris_df_train.iloc[:,-1].values
y_train

array([2, 1, 2, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 2, 1, 2, 2, 2, 0, 2, 2,
       0, 2, 2, 0, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0,
       0, 2, 0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2,
       0, 2, 2, 0, 0, 1, 0, 2, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1,
       1, 0, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       2, 1, 0, 2, 0, 1, 1, 0, 0, 1], dtype=int64)

In [436]:
# SETTING UP HYPERPARAMETERS AND DATA SET PARAMETERS
num_classes = 3 # -- Iris Setosa -- Iris Versicolour -- Iris Virginica
num_features = 4 # 1. sepal length in cm 2. sepal width in cm  3. petal length in cm 4. petal width in cm

# Training parameters.
learning_rate = .001
training_steps = 2000
batch_size = 10
display_step = 200

# LOADING AND PREPARING THE MNIST DATA SET
x_train,y_train, x_test,y_test = getTestandTrainSplit()
    
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

y_train, y_test = np.array(y_train, np.int64), np.array(y_test, np.int64)

# Normalize images value from [0, 255] to [0, 1].
#x_train, x_test = x_train / 255., x_test / 255.

In [437]:
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

print(x_test.shape,x_train.shape,num_features)

# Use tf.data API to shuffle and batch data.
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

#prefetch: Creates a `Dataset` that prefetches elements from this dataset. 
#Like other `Dataset` methods, prefetch operates on the #elements of the input dataset. 
#It has no concept of examples vs. batches. `examples.prefetch(2)` will prefetch 
#two elements (2 examples), #while `examples.batch(20).prefetch(2)` will prefetch 
#2 elements (2 batches, of 20 examples each).

print(train_data)
print(train_data.repeat())
#Signature: train_data.shuffle(buffer_size, seed=None, reshuffle_each_iteration=None)
train_data=train_data.shuffle(500).repeat().batch(batch_size).prefetch(1)
#Look at https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset
#to understand details of how shuffle(), repeat() and batch() work.

(30, 4) (120, 4) 4
<TensorSliceDataset shapes: ((4,), ()), types: (tf.float32, tf.int64)>
<RepeatDataset shapes: ((4,), ()), types: (tf.float32, tf.int64)>


In [438]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(120, 4) (30, 4) (120,) (30,)


In [451]:
# INITIALIZING WEIGHTS AND BIASES
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")


# DEFINING LOGISTIC REGRESSION AND COST FUNCTION
# Logistic regression (Wx + b).
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# DEFINING OPTIMIZERS AND ACCURACY METRICS
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

# OPTIMIZATION PROCESS AND UPDATING WEIGHTS AND BIASES
def run_optimization(x, y):
# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))
    
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    #print(batch_x,batch_y)
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
    if step % 1000 ==0:
        print(x_test.shape,y_test.shape)
        print("Test Accuracy: %f" % accuracy(logistic_regression(x_test), y_test))

step: 200, loss: 6.353139, accuracy: 0.600000
step: 400, loss: 3.540609, accuracy: 0.900000
step: 600, loss: 4.800160, accuracy: 0.900000
step: 800, loss: 3.297410, accuracy: 1.000000
step: 1000, loss: 2.928525, accuracy: 1.000000
(30, 4) (30,)
Test Accuracy: 1.000000
step: 1200, loss: 4.377698, accuracy: 0.800000
step: 1400, loss: 1.895106, accuracy: 1.000000
step: 1600, loss: 2.759004, accuracy: 0.800000
step: 1800, loss: 1.590866, accuracy: 1.000000
step: 2000, loss: 4.006105, accuracy: 0.900000
(30, 4) (30,)
Test Accuracy: 0.933333


In [452]:
# Test model on test set.
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))


Test Accuracy: 0.933333


In [426]:
# Test model on test set.
pred = logistic_regression(x_train)
print("Test Accuracy: %f" % accuracy(pred, y_train))

Test Accuracy: 0.966667
